In [1]:
# Libraries

import pandas as pd
import geopandas as gpd
import numpy as np
import math

import matplotlib
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from shapely.geometry import Polygon
from sklearn.cluster import DBSCAN

import operator

import random 

In [123]:
data18 = pd.read_csv('../mexico/HH/2018_product_country.csv')

filter2 = pd.read_csv('../mexico/HH/filter2.csv')

In [4]:
# data18 = data18.head(10000)

In [124]:
data18["value_sqaure"] = data18['value'] ** 2

data18_grouped = (data18.groupby(['country_id', 'product_id']))

data18_grouped_square_sum = data18_grouped['value_sqaure', 'value'].agg('sum').reset_index()

data18_grouped_square_sum['concentration'] = data18_grouped_square_sum["value_sqaure"] / (data18_grouped_square_sum["value"])**2


In [125]:
data18_grouped_square_sum

,country_id,product_id,value_sqaure,value,concentration
0,4,10121,9.631690e+14,31122917.0,0.994356
1,4,10594,1.017799e+15,32677085.0,0.953181
2,4,10619,1.248359e+08,11173.0,1.000000
3,4,10632,2.310096e+08,15199.0,1.000000
4,4,20622,6.096754e+13,10262062.0,0.578934
5,4,30119,3.379715e+08,18384.0,1.000000
6,4,30384,1.570210e+14,13573930.0,0.852210
7,4,40210,9.905362e+14,85685584.0,0.134913
8,4,40291,9.302692e+14,44238500.0,0.475344
9,4,40310,1.883667e+14,13743703.0,0.997233


In [126]:
data18_grouped_square_sum.to_csv('Market_concentration_index.csv')


In [127]:
classification = pd.read_csv('../mexico/HH/calificacion.csv')

In [128]:
classification

,country_id,product_id,short_term,large_term,market_size,score
0,4,40210,1,1,0,3
1,4,40291,1,1,1,7
2,4,40310,1,1,0,3
3,4,40610,1,1,0,3
4,4,40900,1,1,0,3
5,4,90210,1,1,1,7
6,4,90230,1,1,1,7
7,4,110100,1,1,1,7
8,4,121490,1,1,0,3
9,4,151800,1,1,1,7


In [129]:
joined_result = pd.merge(classification,
                 data18_grouped_square_sum,
                 how = 'left',
                 on=['country_id','product_id'])

joined_result['data_18'] = joined_result['concentration'].notnull()  * 1


joined_result

,country_id,product_id,short_term,large_term,market_size,score,value_sqaure,value,concentration,data_18
0,4,40210,1,1,0,3,9.905362e+14,8.568558e+07,0.134913,1
1,4,40291,1,1,1,7,9.302692e+14,4.423850e+07,0.475344,1
2,4,40310,1,1,0,3,1.883667e+14,1.374370e+07,0.997233,1
3,4,40610,1,1,0,3,2.834348e+12,3.889723e+06,0.187334,1
4,4,40900,1,1,0,3,3.607880e+10,2.891110e+05,0.431641,1
5,4,90210,1,1,1,7,8.324681e+15,1.210118e+08,0.568476,1
6,4,90230,1,1,1,7,2.292077e+16,1.712089e+08,0.781946,1
7,4,110100,1,1,1,7,1.125133e+18,1.528949e+09,0.481302,1
8,4,121490,1,1,0,3,1.068459e+14,1.988829e+07,0.270124,1
9,4,151800,1,1,1,7,3.025612e+17,7.892449e+08,0.485724,1


In [170]:

# null_loc_list = joined_result.loc[joined_result['2018_data'] == 0].index.tolist()

# null_loc_list

# joined_result = joined_result.loc[joined_result['filter3_result'] == 1]

acc = 1

for i in range (288002):
    if joined_result['data_18'].loc[i]==0:
        
        country_id = joined_result['country_id'].loc[i]
        
        product_id = joined_result['product_id'].loc[i]
        
#         print (country_id, product_id)
        
        acc += 1 
        
        result = data17_grouped_square_sum['concentration'][data17_grouped_square_sum['country_id'] == country_id][data17_grouped_square_sum['product_id'] == product_id].iloc[0]
        
        print (result)
        
#         print (joined_result['concentration'].loc[i])
        
#         print (joined_result['concentration'].loc[i] = data17_grouped_square_sum['concentration'][data17_grouped_square_sum['country_id'] == country_id][data17_grouped_square_sum['product_id'] == product_id].iloc[0]) 

#         print (i)

print (acc)

0.5249562187803722
1.0
1.0
1.0


IndexError: single positional indexer is out-of-bounds

In [97]:
data17 = pd.read_csv('../mexico/HH/2017_product_country.csv')

data17["value_sqaure"] = data17['value'] ** 2

data17_grouped = (data17.groupby(['country_id', 'product_id']))

data17_grouped_square_sum = data17_grouped['value_sqaure', 'value'].agg('sum').reset_index()

data17_grouped_square_sum['concentration'] = data17_grouped_square_sum["value_sqaure"] / (data17_grouped_square_sum["value"])**2

data17_grouped_square_sum

,country_id,product_id,value_sqaure,value,concentration
0,8,10129,2.417403e+08,15548.0,1.000000
1,8,10229,2.387409e+13,8403740.0,0.338051
2,8,10392,1.447838e+14,15325271.0,0.616458
3,8,10511,2.024959e+12,2135844.0,0.443892
4,8,10512,5.050092e+09,71064.0,1.000000
5,8,10594,1.276337e+12,1129751.0,1.000000
6,8,10599,1.176037e+11,342934.0,1.000000
7,8,20230,4.925443e+11,701815.0,1.000000
8,8,20322,3.231718e+12,1797698.0,1.000000
9,8,20329,1.056375e+13,4135854.0,0.617572


In [176]:
def find_concen17(c,p):
        if (c in data17_grouped_square_sum['country_id'].values) and (p in data17_grouped_square_sum['product_id'].loc[].):
            
            print (1)
            
find_concen17(894, 999999)

1


In [148]:
acc = 0

def find_data17(country_id, product_id):
    global acc 
    
    acc += 1
    
    if acc % 1000 == 0:
    
        print (acc // 1000)
        
    return (data17_grouped_square_sum['concentration'][data17_grouped_square_sum['country_id'] == country_id][data17_grouped_square_sum['product_id'] == product_id].iloc[0])

print (find_data17(8,999999))


0.05053665915703063


In [159]:
# joined_result['concentration'].loc[null_loc_list] = joined_result.loc[null_loc_list][['country_id','product_id']].apply(lambda x: find_data17(*x), axis=1)

# joined_result

for i in null_loc_list:
    result = find_data17(data17_grouped_square_sum['country_id'].loc[i],data17_grouped_square_sum['product_id'].loc[i])
    
    print (result)
    
    data17_grouped_square_sum['concentration'].loc[i] = 

0.9805712544849018
1.0
0.5033144221351732
0.8331984959933236
0.9866109989211982
1.0
1.0
0.9992989854459638
1.0
1.0
1.0
0.5799103789730848
0.3969144941973134
0.9093206075953022
1.0
0.5374721792791635
0.9424797271760291
1.0
0.9085077271511425
1.0
0.94615105501607
0.7365192431185227
0.7999047672124836
0.540227831262925
0.6721670835859154
0.6529363593782512
0.47899244456335005
0.7215626265025231
0.5254080485617201
0.8191925244392491
1.0
0.41716402809495073
1.0
1.0
0.3923767248123233
1.0
1.0
0.47378260611085293
0.24149833723536407
0.676102481273479
1.0
1.0
1.0
1.0
1.0
0.2963906437528837
0.5133791373647631
0.40215233261126815
1.0
0.3828087558376464
1.0
0.5646100932330594
0.5191807328666099
0.4670235054192537
0.3493771230638754
0.3254813512218709
0.5403596935968974
1.0
0.47988045568431537
0.35729056405801296
0.49884035251362585
0.42229503157748444
1.0
0.9486268855816443
1.0
1.0
1.0
0.5201541118877034
0.8978437726303155
1.0
1.0
0.8077399184148324
0.5566935822777144
1.0
1.0
0.5841291704909033
0

0.631364869206952
1.0
0.4353702597865001
1.0
0.44838901776017825
0.7385399848855934
1.0
1.0
0.3574856432933921
1.0
0.9464530486498326
1.0
0.380546155558037
1.0
0.9456813425612262
0.3967461675624875
0.28790880147906345
0.8264476765657836
1.0
0.5652819827774949
1.0
0.534424681572917
0.3778994111550078
0.40100694400552694
0.44588298541502575
0.47575714774281525
0.4737210785496632
0.7000127766168723
1.0
0.5157208665057872
1.0
0.45822222480317537
0.5252313435826317
0.7428269952865801
0.7736961414902402
0.7269151817155084
0.37013482360961947
0.42882120488164893
0.29306447803454705
0.4729691069662572
0.7755463082489881
1.0
0.469648469154233
0.49391792155716513
1.0
0.2798966483134938
1.0
0.813648752036468
1.0
1.0
1.0
1.0
1.0
0.47032813648306826
0.49399212585990754
0.426033963746203
0.8436085915138719
0.5689419181963886
0.30144365919010674
0.8763030526209141
0.34121827047888387
1.0
0.5383945085854185
1.0
0.8613160256052049
0.7534152687287211
0.8463869132529074
1.0
0.7649378575090023
0.996452590

0.881260244384079
0.9064542333525383
1.0
1.0
1.0
1.0
0.5166430621823638
0.2252633221606903
0.5113189692392808
1.0
0.580091770719967
0.3044587203773711
0.3805477808440772
0.34516475259707113
0.3185448960948682
0.6339368962731629
0.4414744580660888
0.6603453980069184
0.34549905520853047
0.1926594358445611
0.36657759972634507
0.8091664483787641
0.40163533646859756
0.21521522299899792
0.28603042673075635
0.49884417267763886
0.3436077372386139
0.963336275722351
0.9635716427768294
0.26828293855168106
0.2624939104808997
0.36758241751855497
0.6510942989683453
0.8661726863038006
0.35104960540032226
1.0
0.2556444936089245
0.6989052122127035
0.6652055318207388
0.3530168986075616
0.25222043845481523
0.883161262944585
1.0
0.8203249110765178
1.0
0.7897560833894859
0.8565590030765298
0.9308976338406905
1.0
1.0
1.0
1.0
1.0
0.9376247913543487
1.0
1.0
0.436112194466994
0.3850933925732646
1.0
1.0
0.3411570106058118
0.6371539690867764
0.7329967754096955
0.3311699236996077
0.35556486054879294
0.31174903821

0.42050054878796217
0.24471811064703003
0.494820734972701
1.0
0.9029751859097475
0.3412730963452315
1.0
0.707674987893788
0.1501259454158348
0.893539117517243
0.9268404385452055
0.9377297160437172
0.17749226060372317
0.23355741072630312
0.3219985193723381
0.3879264513429236
0.830076663131913
0.14377755073189669
0.335863419483003
0.16015517165847043
0.2779629565898691
0.14570476518695177
0.1911967806944264
0.8596782881313902
0.4624702278554717
0.2790032126384268
0.16373724755571614
0.8892694994288335
0.6391383759005768
0.6388651306351651
0.30738203970753436
0.6954141555681238
0.6089526265451896
0.35465060065628956
0.9757534910467208
0.8669071011272681
0.2990922839162642
0.18241004782159573
0.37747958730383674
0.14492119468394474
0.2696764259231864
0.17162759830783395
0.2987551403395819
0.5190362879238548
1.0
0.20088158795769087
0.5077686615872774
1.0
1.0
0.8770375082491357
0.9004207278937817
0.33865216424542416
0.5055125
1.0
1.0
1.0
0.45535498346363357
1.0
1.0
0.33104947388996914
0.3831

0.5991492025538034
0.4510736948995061
0.6627729492729562
0.6365857744395628
0.5028321301592716
0.35448976638894725
0.2887459760482751
1.0
0.8276361088843035
0.5178297565992787
0.5230459504220912
0.5482452457296433
0.9155322600562518
0.4997041538729812
0.9927940296017774
0.7930849256793613
1.0
0.9147781857132434
0.9393053916286485
1.0
0.8374078519568418
0.5089602793088679
0.8253669354750274
0.6767522809553466
0.6594834858026065
0.6006153940319896
0.5848161691799552
0.5970501674589266
0.46517126583241714
0.3542263340618949
0.6049551756092392
0.4777512610699424
0.6721834491482384
0.859137378312052
0.43518484602334184
1.0
0.6294307109039379
1.0
0.7432285025560522
0.965922945467783
1.0
0.5939084888865159
0.8006223507849499
0.5724590944186021
0.9039859307920185
1.0
0.8850404373374828
0.30785710423875434
1.0
0.39395170729583795
0.5834003455635661
0.35752258675172743
0.39061089015758577
0.5320578037216799
0.6412607515537977
0.25739248895233613
0.512843487456054
0.39739450147136174
0.6737272276

...

0.9426156491554826
0.4807729159140226
0.8110553198879857
0.9780239833392879
0.6761136214418298
0.32511748983482114
0.3545014633542483
0.45975713458735123
0.5016192314376893
0.8254783722347145
1.0
1.0
0.2870723414821115
0.8643038573021071
1.0
0.9098589744764438
0.6633407356459153
0.33079119419365305
0.7677666384376005
0.21779132416105218
0.291919107142573
0.3753410659787321
0.6095673226927626
0.43040220692441494
0.26755282478912845
0.19211234141487232
0.5125423812473634
0.240304271741947
0.5331999569901256
0.6513748796376423
0.2760724877277876
1.0
1.0
0.5998687377738378
1.0
1.0
0.6562892458192875
0.5743504084382964
1.0
0.8557896778301051
0.46419920340480664
0.9972476722845525
0.6783331590319824
0.4366361861463071
0.7172702983260486
0.5769301597420946
0.3268369349757176
0.44714456011345977
1.0
0.8538674729150919
0.7479604312372545
1.0
1.0
0.9439491568916687
1.0
0.6790336738742313
1.0
0.8221316530087319
1.0
1.0
0.5139825534055159
1.0
1.0
1.0
1.0
1.0
0.5000032051467079
1.0
1.0
1.0
1.0
0.51

0.9538401462162588
0.35249215223349545
0.25834763483134005
0.41378848095220905
1.0
0.38576476101445933
0.17880787175510393
0.2354995068334901
0.30732901698064224
0.1802130459651104
0.36549323650226545
0.41942698613851287
0.3872047289229991
0.8125832510740373
0.6298273013133351
0.23935841204793842
0.3117592410490734
0.8631921769582507
0.41406083829320267
0.7709466077537788
0.18693127694527564
0.21441299338407893
0.3398596635341102
0.5097032306536439
0.40007644462698916
0.5176311910075552
0.48883401920438957
0.3235953106869095
0.5263207211259159
0.26199072142029944
0.22088612519533612
0.3222664703815664
0.3554935393803917
0.4003898043047457
0.9464728913881807
0.3023307003946293
0.2751928792894029
0.43203679531659167
0.20252421652343905
0.4813569052294625
0.4892458285781933
0.598077040915963
0.2776226398028059
0.8190458552983765
0.2667151412356377
0.552381369109661
0.7142959742459435
0.4641715044666361
0.3323743482393842
0.3965683027521183
0.38767710601862326
0.24470818141992487
0.2019366

1.0
1.0
0.65480777264708
0.2487608047186799
0.7797762088039987
0.7563909753922089
0.4811759028013605
0.21125516526569457
0.2608548219949412
0.3577523151932688
0.2129875456140849
0.36035072811401486
0.28816307438475475
0.43287042777739043
0.3052280439815605
0.20591104387296832
0.2617542079172946
0.2477773357568142
0.20991474731038023
0.3385459189215624
0.4101705051804396
0.24820365605047812
0.2459918060948403
0.36283172739292496
0.2431357184730635
0.30424576238100237
0.46202711860952156
0.5366746988181686
0.20746591808042508
0.5194276364059409
0.500179124083233
0.24302152204295005
0.5116791419046958
0.25370251781794134
0.9146751094901815
0.5116341747789223
0.5216523573961509
0.39346344626110896
0.5495979248407223
0.500734608266485
0.7466004991470825
0.7024510119997387
0.7117843174677386
0.2484768925026527
0.3871039476142087
0.31029267788083303
0.865995157258288
0.8367657550535077
0.6587249548986059
1.0
0.7749181203275187
0.46180832903422925
0.6469388081408379
0.2836005335981226
1.0
0.32

0.24669396585438141
0.46797853372784165
0.3255919531804665
0.3802793913130022
0.905137622928853
0.9763882961783493
0.3254163979906663
0.43632368348049066
0.30583713528938505
0.5010697715065096
0.403864545013938
0.9686142556009926
1.0
0.973651422209274
0.9935359571741189
1.0
0.6096072999596507
1.0
0.8919013131720062
0.43729727208723657
0.9092218838511231
0.7505083821988658
0.9642401599245045
0.9832796613247664
0.9642842870412035
0.9701338080017375
0.4730020182152951
0.2041906566891498
0.28582129089353353
0.46407127092243083
0.32586522380818134
0.2647881137847295
0.5285277721696132
0.6671524059152809
0.42527622894509987
0.24639471985814965
0.3271156032189926
0.22167234028392452
0.24797616948980664
0.23289570505127916
0.17170196624510473
0.5355730067762399
0.663659930822059
0.7787675567512515
0.2655895129606948
0.3167146157175854
0.5952255618267589
0.21086724253270978
0.36466601479071326
0.716876103763019
0.30407351074196337
0.7570943096321503
0.13315465537663548
0.41970369773965105
0.278

KeyboardInterrupt: 

In [155]:
new = joined_result.loc[null_loc_list]

new 
#  = joined_result.loc[null_loc_list][['country_id','product_id']].apply(lambda x: find_data17(*x), axis=1)


,country_id,product_id,short_term,large_term,market_size,score,value_sqaure,value,concentration,data_18
44,8,20727,1,1,0,3,NaN,NaN,NaN,0
70,8,70992,1,1,0,3,NaN,NaN,NaN,0
112,8,110419,1,1,0,3,NaN,NaN,NaN,0
117,8,120921,1,1,0,3,NaN,NaN,NaN,0
120,8,121490,1,1,0,3,NaN,NaN,NaN,0
121,8,130239,1,1,0,3,NaN,NaN,NaN,0
131,8,152190,1,1,0,3,NaN,NaN,NaN,0
165,8,200950,1,1,0,3,NaN,NaN,NaN,0
172,8,220290,1,1,0,3,NaN,NaN,NaN,0
175,8,220840,1,1,0,3,NaN,NaN,NaN,0


In [151]:
joined_result.loc[null_loc_list]

,country_id,product_id,short_term,large_term,market_size,score,value_sqaure,value,concentration,data_18
44,8,20727,1,1,0,3,NaN,NaN,NaN,0
70,8,70992,1,1,0,3,NaN,NaN,NaN,0
112,8,110419,1,1,0,3,NaN,NaN,NaN,0
117,8,120921,1,1,0,3,NaN,NaN,NaN,0
120,8,121490,1,1,0,3,NaN,NaN,NaN,0
121,8,130239,1,1,0,3,NaN,NaN,NaN,0
131,8,152190,1,1,0,3,NaN,NaN,NaN,0
165,8,200950,1,1,0,3,NaN,NaN,NaN,0
172,8,220290,1,1,0,3,NaN,NaN,NaN,0
175,8,220840,1,1,0,3,NaN,NaN,NaN,0


In [ ]:
joined_result

In [ ]:
for item in null_loc_list:
    data17_grouped_square_sum['concentration'].loc[item] = data17_grouped_square_sum['']

In [12]:
def filter_3(cate, concen):
    if (cate == 3 and concen < 0.4) or ((cate == 4 or cate == 5 or cate == 6) and concen < 0.5) or (cate == 7 and concen < 0.6):
        return (1)
    
    else:
        return (0)

    
joined_result['filter3_result'] = joined_result[['score','concentration']].apply(lambda x: filter_3(*x), axis=1)


In [13]:
joined_result

,country_id,product_id,short_term,large_term,market_size,score,value_sqaure,value,concentration,filter3_result
0,4,40210,1,1,0,3,9.905362e+14,8.568558e+07,0.134913,1
1,4,40291,1,1,1,7,9.302692e+14,4.423850e+07,0.475344,1
2,4,40310,1,1,0,3,1.883667e+14,1.374370e+07,0.997233,0
3,4,40610,1,1,0,3,2.834348e+12,3.889723e+06,0.187334,1
4,4,40900,1,1,0,3,3.607880e+10,2.891110e+05,0.431641,0
5,4,90210,1,1,1,7,8.324681e+15,1.210118e+08,0.568476,1
6,4,90230,1,1,1,7,2.292077e+16,1.712089e+08,0.781946,0
7,4,110100,1,1,1,7,1.125133e+18,1.528949e+09,0.481302,1
8,4,121490,1,1,0,3,1.068459e+14,1.988829e+07,0.270124,1
9,4,151800,1,1,1,7,3.025612e+17,7.892449e+08,0.485724,1


In [14]:
joined_result.to_csv('filter3_result.csv')

In [15]:
result_NaN = pd.merge(classification,
                 data18_grouped_square_sum,
                 how = 'left',
                 on=['country_id','product_id'])

result_NaN = result_NaN[result_NaN['concentration'].isnull()]


In [16]:
result_NaN.to_csv('Null_Rows.csv')

In [18]:
joined_result = joined_result.loc[joined_result['filter3_result'] == 1]

In [19]:
joined_result

,country_id,product_id,short_term,large_term,market_size,score,value_sqaure,value,concentration,filter3_result
0,4,40210,1,1,0,3,9.905362e+14,8.568558e+07,0.134913,1
1,4,40291,1,1,1,7,9.302692e+14,4.423850e+07,0.475344,1
3,4,40610,1,1,0,3,2.834348e+12,3.889723e+06,0.187334,1
5,4,90210,1,1,1,7,8.324681e+15,1.210118e+08,0.568476,1
7,4,110100,1,1,1,7,1.125133e+18,1.528949e+09,0.481302,1
8,4,121490,1,1,0,3,1.068459e+14,1.988829e+07,0.270124,1
9,4,151800,1,1,1,7,3.025612e+17,7.892449e+08,0.485724,1
12,4,240220,1,1,0,3,3.314218e+15,1.032007e+08,0.311183,1
14,4,270300,0,0,1,4,6.505113e+17,1.252496e+09,0.414669,1
16,4,271121,1,1,0,3,5.523514e+16,4.271114e+08,0.302784,1


In [21]:

joined_result_grouped = (joined_result.groupby(['product_id']))

joined_result_grouped_sum = joined_result_grouped['filter3_result'].agg('sum').reset_index()

# all_data_grouped_square_sum['concentration'] = all_data_grouped_square_sum["value_sqaure"] / (all_data_grouped_square_sum["value"])**2


In [23]:
joined_result_grouped_sum

,product_id,filter3_result
0,10121,27
1,10129,12
2,10130,2
3,10190,4
4,10221,17
5,10229,5
6,10290,2
7,10310,9
8,10410,8
9,10420,5


In [ ]:
joined_result_grouped_sum.to_csv('Country-No. Recommendation.csv')

In [24]:
joined_result

,country_id,product_id,short_term,large_term,market_size,score,value_sqaure,value,concentration,filter3_result
0,4,40210,1,1,0,3,9.905362e+14,8.568558e+07,0.134913,1
1,4,40291,1,1,1,7,9.302692e+14,4.423850e+07,0.475344,1
3,4,40610,1,1,0,3,2.834348e+12,3.889723e+06,0.187334,1
5,4,90210,1,1,1,7,8.324681e+15,1.210118e+08,0.568476,1
7,4,110100,1,1,1,7,1.125133e+18,1.528949e+09,0.481302,1
8,4,121490,1,1,0,3,1.068459e+14,1.988829e+07,0.270124,1
9,4,151800,1,1,1,7,3.025612e+17,7.892449e+08,0.485724,1
12,4,240220,1,1,0,3,3.314218e+15,1.032007e+08,0.311183,1
14,4,270300,0,0,1,4,6.505113e+17,1.252496e+09,0.414669,1
16,4,271121,1,1,0,3,5.523514e+16,4.271114e+08,0.302784,1


In [25]:
ranking = pd.read_csv('../mexico/HH/filtro2.csv')

In [31]:
ranking

,country_id,product_id,country_idz,ranking
0,484,10110,0,1
1,484,10110,0,2
2,484,10110,0,3
3,484,10110,0,4
4,484,10110,0,5
5,484,10110,0,6
6,484,10110,0,7
7,484,10110,0,8
8,484,10110,0,9
9,484,10110,0,10


In [70]:
def find_ranked_country_list (product_id):

    choose1 = ranking.loc[ranking['product_id'] == product_id].loc[ranking['country_idz'] != 0]

    choose1 = choose1.sort_values("ranking", inplace = False) 

    ranked_index_list = choose1.index.tolist()

    output_list = choose1['country_idz'].loc[ranked_index_list].tolist()

    return (output_list)

In [73]:
output = pd.DataFrame()

output['product_id'] = (ranking['product_id'].unique())

output['ranked_list'] = output['product_id'].apply(lambda x: find_ranked_country_list(x))

output

,product_id,ranked_list
0,10110,[]
1,10121,"[842, 124, 392, 152, 604, 826, 76, 372, 251, 5..."
2,10129,"[842, 124, 392, 344, 36, 152, 826, 372, 251, 7..."
3,10130,"[842, 156, 56, 381, 214, 764]"
4,10190,"[251, 56, 276, 528, 643]"
5,10210,[]
6,10221,"[792, 156, 704, 360, 381, 643, 818, 320, 392, ..."
7,10229,"[842, 792, 156, 400, 340, 643, 320, 724, 764, ..."
8,10231,[764]
9,10239,[842]
